In [1]:
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import numpy as np

### For a single book

In [4]:
url_book = 'https://books.toscrape.com/catalogue/the-metamorphosis_409/index.html'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5), AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

page = requests.get(url_book, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')

In [5]:
book_info = soup.find('div', class_='col-sm-6 product_main')

book_name = book_info.find('h1').get_text()

# dropping the first character because it's garbage
book_price = book_info.find('p', class_='price_color').get_text()[1:]

# I am using filter to get only value characters
# Then splitting using \n, getting thea second element, which is the stock
# Then removing the white spaces before the stock
book_stock = list(filter(None, book_info.find('p',
                         class_='instock availability').get_text().split('\n')))[1].strip()

# here I am getting all the classes for p and then selecting the last element
book_review = [value for element in book_info.find_all('p', class_=True) for value in element["class"]][-1]

### For a single category

In [6]:
url = 'https://books.toscrape.com/catalogue/category/books/classics_6/index.html'

# the header is added so the request is treated like a web browser
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5), AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')

In [7]:
books = soup.find('ol', class_='row')
links = books.find_all('div', class_='image_container')

full_links = []

for l in links:
    # building the URLs, removing the first 9 characters of the link since it's gargabe
    url = 'https://books.toscrape.com/catalogue/' + str(l.find('a', href=True)['href'][9:])
    full_links.append(url)

In [8]:
book_names = []
book_prices = []
book_reviews = []
book_stocks = []
n = 0

for i in full_links:
    
    print('Book number: {0}'.format(n))
    n += 1
    url = i

    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    book_info = soup.find('div', class_='col-sm-6 product_main')

    book_name = book_info.find('h1').get_text()
    book_names.append(book_name)

    # dropping the first character because it's garbage
    book_price = book_info.find('p', class_='price_color').get_text()[1:]
    book_prices.append(book_price)

    # I am using filter to get only value characters
    # Then splitting using \n, getting thea second element, which is the stock
    # Then removing the white spaces before the stock
    book_stock = list(filter(None, book_info.find('p',
                         class_='instock availability').get_text().split('\n')))[1].strip()
    book_stocks.append(book_stock)

    # here I am getting all the classes for p and then selecting the last element
    book_review = [value for element in book_info.find_all('p', class_=True) for value in element["class"]][-1]
    book_reviews.append(book_review)

Book number: 0
Book number: 1
Book number: 2
Book number: 3
Book number: 4
Book number: 5
Book number: 6
Book number: 7
Book number: 8
Book number: 9
Book number: 10
Book number: 11
Book number: 12
Book number: 13
Book number: 14
Book number: 15
Book number: 16
Book number: 17
Book number: 18


In [9]:
books_df = pd.DataFrame([book_names, book_prices, book_reviews, book_stocks]).T
books_df.columns = [['Name', 'Price', 'Stock', 'Review']]
books_df.head()

,Name,Price,Stock,Review
0,The Secret Garden,£15.08,Four,In stock (5 available)
1,The Metamorphosis,£28.58,One,In stock (5 available)
2,The Pilgrim's Progress,£50.26,Two,In stock (4 available)
3,The Hound of the Baskervilles (Sherlock Holmes...,£14.82,Two,In stock (4 available)
4,Little Women (Little Women #1),£28.07,Four,In stock (4 available)


### For multiple categories

In [10]:
url = 'https://books.toscrape.com/index.html'

# the header is added so the request is treated like a web browser
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5), AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
categories = soup.find_all('ul', class_='nav nav-list')
categories_links = []
for i in categories:
    all_links = i.find_all('a', href=True)
    for l in all_links:
        url = 'https://books.toscrape.com/' + l['href']
        categories_links.append(url)

# getting books only for four categories
filtered_links = []
filters = ['classics_6', 'science-fiction_16', 'humor_30', 'business_35']
# for loop in all links
for l in categories_links:
    # for loop in the filters
    for f in filters:
        # if the filter is present in the link, add the link to the list
        if f in l:
            filtered_links.append(l)

In [ ]:
category = ''
for link in filtered_links:
    if 'classics_6' in link:
        category = 'Classics'
    elif 'science-fiction_16' in link:
        category = 'Science Fiction'
    elif 'humor_30' in link:
        category = 'Humor'
    else:
        category = 'Business'
        
category

In [ ]:
data_raw = pd.DataFrame()

n = 0

for link in filtered_links:
    
    url = link

    # the header is added so the request is treated like a web browser
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5), AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    books = soup.find('ol', class_='row')
    links = books.find_all('div', class_='image_container')

    full_links = []

    for l in links:
        # building the URLs, removing the first 9 characters of the link since it's gargabe
        url = 'https://books.toscrape.com/catalogue/' + str(l.find('a', href=True)['href'][9:])
        full_links.append(url)
        
    book_names = []
    book_prices = []
    book_reviews = []
    book_stocks = []
    book_category = []
    
    # added this if clauses here to change the category
    if 'classics_6' in link:
        category = 'Classics'
    elif 'science-fiction_16' in link:
        category = 'Science Fiction'
    elif 'humor_30' in link:
        category = 'Humor'
    else:
        category = 'Business'

    for i in full_links:
        url = i
        
        print('Book number: {0}'.format(n))
        n += 1

        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, 'html.parser')
    
        book_info = soup.find('div', class_='col-sm-6 product_main')

        book_name = book_info.find('h1').get_text()
        book_names.append(book_name)

        # dropping the first character because it's garbage
        book_price = book_info.find('p', class_='price_color').get_text()[1:]
        book_prices.append(book_price)

        # I am using filter to get only value characters
        # Then splitting using \n, getting thea second element, which is the stock
        # Then removing the white spaces before the stock
        book_stock = list(filter(None, book_info.find('p',
                         class_='instock availability').get_text().split('\n')))[1].strip()
        book_stocks.append(book_stock)

        # here I am getting all the classes for p and then selecting the last element
        book_review = [value for element in book_info.find_all('p', class_=True) for value in element["class"]][-1]
        book_reviews.append(book_review)
        
        book_category.append(category)
    
    books_df = pd.DataFrame([book_names, book_prices, book_reviews, book_category, book_stocks]).T
    
    data_raw = pd.concat([data_raw, books_df], axis=0)
    
data_raw.columns = [['Name', 'Price', 'Stock', 'Category', 'Review']]

In [ ]:
data_raw.reset_index(drop=True, inplace=True)

In [ ]:
data_raw

In [ ]:
data_raw.to_csv('books.csv', index=False)